In [1]:

from pathlib import Path

from ReChisel.testcase import Testcase
from ReChisel.chisel_code import ChiselCode
from ReChisel.verifier import VerifyResult, verify


bmcase = Testcase(
    'Prob030_popcount255', 
    'benchmarks/VerilogEval_Prob030/Prob030_popcount255_spec.txt',
    'benchmarks/VerilogEval_Prob030/Prob030_popcount255_ref.sv',
    'benchmarks/VerilogEval_Prob030/Prob030_popcount255_tb.sv'
)
print(bmcase.specification)


I would like you to implement a module named TopModule with the following
interface. All input and output ports are one bit unless otherwise
specified.

 - input  in  (255 bits)
 - output out (  8 bits)

A "population count" circuit counts the number of '1's in an input
vector. The module should implement a population count circuit for a
255-bit input vector.


In [2]:

from ReChisel.generator import Generator
from ReChisel.reviewer import Reviewer
from ReChisel.tracing import Tracing


generator = Generator(
    init_gen_system_prompt=Path('prompts/chisel_generation.txt').read_text(encoding='utf-8'),
    init_gen_model='gpt-4o-mini',
    syntax_correction_system_prompt=Path('prompts/syntax_correction.txt').read_text(encoding='utf-8'),
    functionality_correction_system_prompt=Path('prompts/functionality_correction.txt').read_text(encoding='utf-8'),
    correction_model='gpt-4o-mini',
    verbose=False
)
generator.testcase_prepare(bmcase, 'TopModule')

reviewer = Reviewer(
    sbt_system_prompt=Path('prompts/syntax_sbt_reflection.txt').read_text(encoding='utf-8'),
    iv_system_prompt=Path('prompts/syntax_iv_reflection.txt').read_text(encoding='utf-8'),
    functionality_system_prompt=Path('prompts/functionality_reflection.txt').read_text(encoding='utf-8'),
    model='gpt-4o-mini',
    verbose=False
)

tracing = Tracing(
    bmcase,
    use_llm_summary=True,
    llm_summary_model='gpt-4o-mini',
    llm_summary_system_prompt=Path('prompts/attempt_summary.txt').read_text(encoding='utf-8'),
)


/home/niujuxin/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_arn" in BedrockRerank has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:

from langchain_core.messages import AIMessage


current_chisel_code: ChiselCode = None
current_verify_result: VerifyResult = None
current_reviewer_response: AIMessage = None
attempt_count = 0

while True:
    print(f"Attempt {attempt_count + 1}...")
    if current_reviewer_response is None:
        generation_response = generator.initial_chisel_generation()
    else:
        generation_response = generator.correction_generation(
            current_reviewer_response,
            current_verify_result,
            current_chisel_code
        )
    current_chisel_code = generator.code_extract(generation_response)

    current_verify_result = verify(
        current_chisel_code,
        bmcase,
        output_dir=Path('output/verification'),
        bm_type='verilog-eval',
        verbose=False
    )

    if current_verify_result.functionality_correct:
        print(f"Verification passed after {attempt_count + 1} attempts, stopping the process.")
        break
    else:
        current_reviewer_response = reviewer(bmcase, current_verify_result, current_chisel_code)

        tracing.add_attempt(
            current_chisel_code, 
            current_verify_result, 
            current_reviewer_response
        )

    attempt_count += 1
    if attempt_count >= 3:
        print("Maximum attempts reached, stopping the process.")
        break


Attempt 1...
Attempt 2...
Verification passed after 2 attempts, stopping the process.


In [4]:

for i, attempts in enumerate(tracing.last_k_attempts(-1)):
    print(f"======== Attempt {i + 1} ========")
    print("Chisel Code:")
    print(f"```\n{attempts.chisel_code.raw_stripped}\n```")
    print("Summary:")
    print(attempts.summary)
    print()


======== Attempt 1 ========
Chisel Code:
```
class TopModule extends RawModule {
  // Define input and output ports
  val in = IO(Input(UInt(255.W)))   // 255-bit input
  val out = IO(Output(UInt(8.W)))    // 8-bit output

  // Population count logic
  out := PopCount(in).asUInt(8.W) // Count '1's in the input and restrict to 8 bits
}
```
Summary:
The error in the code arises from incorrectly using `asUInt(8.W)` with the `PopCount` result, which doesn't accept a width parameter. To fix this, change the line to `out := PopCount(in)(7, 0)` to correctly take the lower 8 bits of the population count.



In [5]:
from ReChisel.tracing import in_context_attempt_history_format


ictx_history = in_context_attempt_history_format(tracing, -1)
print(ictx_history.content)


Below are the most recent consecutive k attempts trying to implement this Chisel module. For each attempt, the corresponding code is provided along with a summary of the errors found in that version and the suggested modifications. 

NOTE: Please refer to these past attempts to avoid repeating the same mistakes.

## Attempt 1

Chisel Code (omitted):
```scala
class TopModule extends RawModule {
  // Define input and output ports
  val in = IO(Input(UInt(255.W)))   // 255-bit input
  val out = IO(Output(UInt(8.W)))    // 8-bit output

  // Population count logic
  out := PopCount(in).asUInt(8.W) // Count '1's in the input and restrict to 8 bits
}
```
Summary: The error in the code arises from incorrectly using `asUInt(8.W)` with the `PopCount` result, which doesn't accept a width parameter. To fix this, change the line to `out := PopCount(in)(7, 0)` to correctly take the lower 8 bits of the population count.




In [6]:
print('Corrected Chisel Code:\n')
print(current_chisel_code.raw_stripped)

Corrected Chisel Code:

class TopModule extends RawModule {
  // Define input and output ports
  val in = IO(Input(UInt(255.W)))   // 255-bit input
  val out = IO(Output(UInt(8.W)))    // 8-bit output

  // Population count logic
  out := PopCount(in)(7, 0) // Count '1's in the input and take the lower 8 bits
}
